In [1]:
# default_exp outlier_scoring


# Outlier scoring

The default approach of AlphaQuant is to collect the most consistent set of peptides for protein quantification. Potentially, however, there can be valuable information in non-consistent, i.e. outlier peptides, which can be indicative of proteoforms. In this module, we perform a systematic evaluation of outlying groups of peptides in order to determine promising candidates for further investigation.

In particuar, we:

1) Load the clustered tree data and retrieve all proteins with their clustered child peptides
2) Compare the fold changes of the main cluster with the fold changes of the outlier-clusters
3) Filter by both strength of fc diff and the quality of the score
4) Return ranked list


In [2]:
#export
import alphaquant.diffquant_utils as aqutils
import numpy as np
import copy

class OutlierHandler():
    def __init__(self, results_dir, cond1, cond2):
        self._protnodes = self.__load_protein_nodes__(results_dir, cond1, cond2)

    def __load_protein_nodes__(self, results_dir, cond1, cond2):
        return aqutils.read_condpair_tree(results_folder=results_dir, cond1=cond1, cond2 = cond2).children
    
    def get_diffclust_overview_list(self):
        """_summary_

        Returns:
            ClusterDiffHandler: object containing the relevant information about two differing clusters (fcfc, peptides), 
            can "reduce" protein nodes to the cluster-relevant peptides
        """
        diffclusts = []
        counter = 0
        for protnode in self._protnodes:
            counter+=1
            cluster_checker = ProtnodeClusterChecker(protnode)
            diffclusts += cluster_checker.get_diffclusts()
        return diffclusts


class ProtnodeClusterChecker():
    def __init__(self, protnode):
        self._protnode = protnode
        self._num_clusters = protnode.num_clusters

    def get_diffclusts(self):
        if not self.__check_if_multiple_clusters__():
            return []
        return self.__get_clusterdiff_handler_for_each_cluster__()

    def __get_clusterdiff_handler_for_each_cluster__(self):
        protnodes = []
        mainclust_info= self.__get_cluster_info__(clustnum = 0)
        for clustnum in range(1, self._num_clusters):
            protnodes.append(self.__get_clusterdiff_handler__(clustnum, mainclust_info))
        return protnodes
    
    def __get_clusterdiff_handler__(self, clustnum, mainclust_info):
        outlier_info = self.__get_cluster_info__(clustnum)
        return ClusterDiffHandler(mainclust_info, outlier_info)
    
    def __get_cluster_info__(self, clustnum):
        mainclust_peptides = self.__get_peptides_of_cluster__(clustnum)
        return ClusterInfo(protein_name=self._protnode.name,peptide_nodes = mainclust_peptides)

    def __get_peptides_of_cluster__(self, clustnum):
        return [x for x in self._protnode.children if x.cluster == clustnum]
    
    def __check_if_multiple_clusters__(protein_node):
        return protein_node._num_clusters >1


class ClusterInfo():
    def __init__(self, protein_name,peptide_nodes):
        self.protein_name = protein_name
        self.cluster_number = list({x.cluster for x in peptide_nodes})[0]
        self.peptide_names = [x.name for x in peptide_nodes]
        self.median_fc = np.median(np.array([x.fc for x in peptide_nodes]))
        self.minimum_predscore = min([abs(x.predscore) for x in peptide_nodes])

class ClusterDiffHandler():
    def __init__(self, mainclust_info, outlier_info):
        self.protein_name = mainclust_info.protein_name
        self.clusterpair_id = f"{mainclust_info.cluster_number}_{outlier_info.cluster_number}"
        self.name = f"{self.protein_name}_{self.clusterpair_id}"
        self.fcdiff = abs(mainclust_info.median_fc - outlier_info.median_fc)
        self.quality_score = max(mainclust_info.minimum_predscore, outlier_info.minimum_predscore)
        self.peptide_names = mainclust_info.peptide_names + outlier_info.peptide_names
    
    def get_clusterdiff_protnode(self, protnode):
        protnode_clusterdiff = copy.deepcopy(protnode)
        self.__remove_peptides_not_in_cluster__(protnode_clusterdiff)
        self.__add_diffinfos__(protnode_clusterdiff)
        return protnode_clusterdiff

    def __remove_peptides_not_in_cluster__(self, protnode_clusterdiff):
        for peptide_node in protnode_clusterdiff.children:
            self.__remove_peptide_if_necessary__(peptide_node)

    def __add_diffinfos__(self, protnode):
        protnode.fcdiff = self.fcdiff
        protnode.quality_score = self.quality_score
        protnode.peptide_names = self.peptide_names
    
    def __remove_peptide_if_necessary__(self, peptide_node):
        if peptide_node.name not in self.peptide_names:
            peptide_node.parent = None
    

In [3]:
#hide
import anytree
import unittest

def test_cluster_check():
    
    prot = anytree.Node(name="protein")
    prot.num_clusters = 3
    add_peptide_nodes(prot)
    
    diffclust_annotations = ProtnodeClusterChecker(prot).get_diffclusts()
    fcdiffs = [x.fcdiff for x in diffclust_annotations]
    quality_scores = [x.quality_score for x in diffclust_annotations]
    peptides = [x.peptide_names for x in diffclust_annotations]
    
    fcdiffs_ref = [5, 11.5]
    quality_scores_ref = [0.3, 0.4]
    peptides_ref = [[f"pep{x}" for x in range(5)] + [f"pep{x}" for x in range(5, 7)], [f"pep{x}" for x in range(5)] + [f"pep{x}" for x in range(7, 10)]]
    
    assert sorted(fcdiffs) == sorted(fcdiffs_ref)
    assert sorted(quality_scores) == sorted(quality_scores_ref)
    assert sorted(peptides) == sorted(peptides_ref)
    

def add_peptide_nodes(protein_node):
    names = [f"pep{x}" for x in range(10)]
    fold_changes = [-1.5 for x in range(5)] + [3.5 for x in range(2)] + [10 for x in range(3)]
    clusters = [0 for x in range(5)] + [1 for x in range(2)] + [2 for x in range(3)]
    predscores = [0.1 for x in range(5)] + [0.3 for x in range(2)] + [0.4 for x in range(3)]
    nodes = []
    for idx in range(10):
        pepnode = anytree.Node(names[idx])
        pepnode.fc = fold_changes[idx]
        pepnode.cluster = clusters[idx]
        pepnode.predscore = predscores[idx]
        pepnode.parent = protein_node
        nodes.append(pepnode) 
    return nodes

test_cluster_check()

